In [9]:
# ライブラリの導入
import pandas as pd
import biogeme.biogeme_logging as blog
from biogeme.version import getText # なくても良い（バージョン確認のため導入）
from biogeme.database import Database
from biogeme.expressions import Variable, Beta
from biogeme.biogeme import BIOGEME
from biogeme.models import lognested
from biogeme.nests import OneNestForNestedLogit, NestsForNestedLogit

In [10]:
# biogemeのバージョン確認
print(getText())

getText is deprecated; use get_text instead.


biogeme 3.3.1 [2025-12-13]
Home page: http://biogeme.epfl.ch
Submit questions to https://groups.google.com/d/forum/biogeme
Michel Bierlaire, Transport and Mobility Laboratory, Ecole Polytechnique Fédérale de Lausanne (EPFL)



In [11]:
# マスターデータの読み込み
master_df = pd.read_csv("/home/shibumtk/B4research/estimate/data/01master_data/Logit_master5.csv", encoding="utf-8-sig")

# 欠損している部分をすべて0で補完（必要な処理は作成時にやってるから大丈夫）
master_df = master_df.fillna(0)

# 確認
print(len(master_df))
master_df.columns

49135


Index(['Personal_ID', 'HouseholdIncome', 'sex', 'age', 'JobType',
       'ComuTime[m]', 'MainlineTime[m]', 'AccessTime_used[m]', 'HouseholdType',
       'HouseholdMembers(all)', 'WorkTime[m]', 'first_transportation',
       'NearestStation', 'NearestStation_code', 'near_area_syou',
       'near_area_kinrin', 'near_area_sum', 'near_area_syou500',
       'near_area_kinrin500', 'near_area_sum500', 'near_area_syou300',
       'near_area_kinrin300', 'near_area_sum300', 'near_o_area_syou',
       'near_o_area_kinrin', 'near_o_area_sum', 'near_o_area_syou500',
       'near_o_area_kinrin500', 'near_o_area_sum500', 'near_o_area_syou300',
       'near_o_area_kinrin300', 'near_o_area_sum300', 'WorkplaceStation',
       'WorkplaceStation_code', 'WP_area_syou', 'WP_area_kinrin',
       'WP_area_sum', 'WP_area_syou500', 'WP_area_kinrin500', 'WP_area_sum500',
       'WP_area_syou300', 'WP_area_kinrin300', 'WP_area_sum300',
       'WP_o_area_syou', 'WP_o_area_kinrin', 'WP_o_area_sum',
       'WP_o_are

In [12]:
# 文字列は除外する
drop_cols =['NearestStation', 'WorkplaceStation']
master_df = master_df.drop(columns=drop_cols)
master_df.columns

Index(['Personal_ID', 'HouseholdIncome', 'sex', 'age', 'JobType',
       'ComuTime[m]', 'MainlineTime[m]', 'AccessTime_used[m]', 'HouseholdType',
       'HouseholdMembers(all)', 'WorkTime[m]', 'first_transportation',
       'NearestStation_code', 'near_area_syou', 'near_area_kinrin',
       'near_area_sum', 'near_area_syou500', 'near_area_kinrin500',
       'near_area_sum500', 'near_area_syou300', 'near_area_kinrin300',
       'near_area_sum300', 'near_o_area_syou', 'near_o_area_kinrin',
       'near_o_area_sum', 'near_o_area_syou500', 'near_o_area_kinrin500',
       'near_o_area_sum500', 'near_o_area_syou300', 'near_o_area_kinrin300',
       'near_o_area_sum300', 'WorkplaceStation_code', 'WP_area_syou',
       'WP_area_kinrin', 'WP_area_sum', 'WP_area_syou500', 'WP_area_kinrin500',
       'WP_area_sum500', 'WP_area_syou300', 'WP_area_kinrin300',
       'WP_area_sum300', 'WP_o_area_syou', 'WP_o_area_kinrin', 'WP_o_area_sum',
       'WP_o_area_syou500', 'WP_o_area_kinrin500', 'WP_o_area

In [13]:
# biogeme.databaseに格納
master_db = Database("PTdata", master_df)

In [ ]:
# 変数の定義
# -------------------------
# 1. Variables (列名は要調整)
# -------------------------
# output
CHOICE_ACCESS = Variable("first_transportation")

# input(TT=Travel/Transit Time)
WALK_AV = Variable("walk_av")
WALK_TT = Variable("walk_time[m]")

BICYCLE_AV = Variable("bicycle_av")
BICYCLE_TT = Variable("bicycle_time[m]")
BICYCLE_CO = Variable("bicycle_parking_fee")

BUS_AV   = Variable("bus_av")
BUS_TT   = Variable("bus_time[m]")
BUS_CO   = Variable("bus_cost")
BUS_DIST = Variable("dist_nearest_BusStop[km]")

CAR_AV = Variable("car_av")
CAR_TT = Variable("car_time[m]")
CAR_CO = Variable("car_cost")
# scaled
WALK_TT_SCALED = master_db.define_variable("WALK_TT_SCALED", WALK_TT / 1)

BICYCLE_TT_SCALED = master_db.define_variable("BICYCLE_TT_SCALED", BICYCLE_TT / 1)
BICYCLE_CO_SCALED = master_db.define_variable("BICYCLE_CO_SCALED", BICYCLE_CO / 100)

BUS_TT_SCALED   = master_db.define_variable("BUS_TT_SCALED", BUS_TT / 1)
BUS_CO_SCALED   = master_db.define_variable("BUS_CO_SCALED", BUS_CO / 100)
BUS_DIST_SCALED = master_db.define_variable("BUS_DIST_SCALED", BUS_DIST * 10)

CAR_TT_SCALED = master_db.define_variable("CAR_TT_SCALED", CAR_TT / 1)
CAR_CO_SCALED = master_db.define_variable("CAR_CO_SCALED", CAR_CO / 100)

# 私事トリップ「する/しない」に効かせたい変数
PRITRIP  = Variable("PriTrip") # 0/1をとる目的変数

HHM_all  = Variable("HouseholdMembers(all)") # 世帯人数
FACAILTY = Variable("FacailtyType")          # 目的地施設種類
PURPOSE  = Variable("TripPurpose")           # トリップ目的
COMUTIME = Variable("ComuTime[m]")           # 通勤時間
MACMTIME = Variable("MainlineTime[m]")       # 通勤時間から端末時間を引いたもの，MACM=MAin CoMuTIME
WORKTIME = Variable("WorkTime[m]")           # 就業時間

N_EKI_SCORE    = Variable("near_area_syou300")   # 利用駅土地利用指標（面積×容積率）
N_EKI_A_SCORE  = Variable("near_o_area_syou300") # 利用駅土地利用指標（面積）
WP_EKI_SCORE   = Variable("WP_area_syou300")     # 就業地駅土地利用指標（面積×容積率）
WP_EKI_A_SCORE = Variable("WP_o_area_syou300")   # 就業地駅土地利用指標（面積）

# スケーリング（係数を0.1~10にするように）
COMUTIME_SCALED = master_db.define_variable("COMUTIME_SCALED", COMUTIME / 60) # 1時間当たりの変化
MACMTIME_SCALED = master_db.define_variable("MACMTIME_SCALED", MACMTIME / 60) 
WORKTIME_SCALED = master_db.define_variable("WORKTIME_SCALED", WORKTIME / 60)

N_EKI_SCORE_SCALED    = master_db.define_variable("N_EKI_SCORE_SCALED",   N_EKI_SCORE / 1000000)   # 1,000,000㎡あたり
N_EKI_A_SCORE_SCALED  = master_db.define_variable("N_EKI_A_SCORE_SCALED", N_EKI_A_SCORE / 1000000) # 1,000,000㎡あたり
WP_EKI_SCORE_SCALED   = master_db.define_variable("WP_EKI_SCORE_SCALED",   WP_EKI_SCORE / 1000000)   # 1,000,000㎡あたり
WP_EKI_A_SCORE_SCALED = master_db.define_variable("WP_EKI_A_SCORE_SCALED", WP_EKI_A_SCORE / 1000000) # 1,000,000㎡あたり

# -------------------------
# 2. Parameters
# -------------------------
# 私事 yes/no（識別のため、どちらか一方のASCを固定）
ASC_NO  = Beta('ASC_NO',  0, None, None, 1)  # 固定（基準）
ASC_YES = Beta('ASC_YES', 0, None, None, 0)

B_MACT = Beta('B_MACT', -0.01, None, None, 0)
B_WT   = Beta('B_WT',   -0.30, None, None, 0)
B_HM   = Beta('B_HM',   0, None, None, 0)
B_NES  = Beta('B_NES',  0, None, None, 0)

# 朝モード（識別のため、1つのASCを固定：例としてWALKを固定）
ASC_WALK    = Beta('ASC_WALK',    0, None, None, 1)  # 固定（基準）
ASC_BICYCLE = Beta('ASC_BICYCLE', 0, None, None, 0)
ASC_BUS     = Beta('ASC_BUS',     0, None, None, 0)
ASC_CAR     = Beta('ASC_CAR',     0, None, None, 0)

B_TIME_LS = Beta('B_TIME_LS', 0, None, None, 0)

# -------------------------
# 3. Utilities
# -------------------------
# 上位（私事トリップする）に関わる共通項
V_NO  = ASC_NO
V_YES = (ASC_YES 
         + B_MACT * MACMTIME_SCALED 
         + B_WT   * WORKTIME_SCALED
         + B_HM   * HHM_all 
         + B_NES  * N_EKI_SCORE_SCALED)

# 可用性（「しない」は常に可用とするのが普通）
av = {1: 1, 2: AV_WALK, 3: AV_BIKE, 4: AV_BUS, 5: AV_CAR}

# -------------------------
# 4. Nests (2段階構造)
# -------------------------
# 「私事トリップする」nest に端末交通（2..5）を入れる
private_trip = OneNestForNestedLogit(
    nest_param=MU_PRIVATE,
    list_of_alternatives=[2, 3, 4, 5],
    name='private_trip',
)

# 単独代替（1）は自動的に trivial nest として扱われる :contentReference[oaicite:2]{index=2}
nests = NestsForNestedLogit(choice_set=list(V), tuple_of_nests=(private_trip,))

# -------------------------
# 5. Likelihood & Estimation
# -------------------------
logprob = lognested(V, av, nests, CHOICE)  # NLの対数尤度 :contentReference[oaicite:3]{index=3}

logger = blog.get_screen_logger(level=blog.INFO)

the_biogeme = BIOGEME(database, logprob, optimization_algorithm='simple_bounds_BFGS')
the_biogeme.model_name = 'two_stage_privateTrip_terminalMode_NL'

the_biogeme.calculate_null_loglikelihood(av)
results = the_biogeme.estimate()

print(results.short_summary())
